In [ ]:
!pip install gdown

In [4]:
! gdown "1Kqf7-0Jkp1doGJWyhgxW1ku738IO4xji"
! gdown "1x_D1Feh88EFHQ63SEe55sAymUnztqzeK"
! gdown "1CcvUQxEuzPbwLOYjA1rxmyBxMF_au6wZ"


Downloading...
From: https://drive.google.com/uc?id=1Kqf7-0Jkp1doGJWyhgxW1ku738IO4xji
To: /content/test.pkl
100% 762k/762k [00:00<00:00, 146MB/s]
Downloading...
From: https://drive.google.com/uc?id=1x_D1Feh88EFHQ63SEe55sAymUnztqzeK
To: /content/train.pkl
100% 3.59M/3.59M [00:00<00:00, 223MB/s]
Downloading...
From: https://drive.google.com/uc?id=1CcvUQxEuzPbwLOYjA1rxmyBxMF_au6wZ
To: /content/val.pkl
100% 769k/769k [00:00<00:00, 94.1MB/s]


In [5]:
import numpy as np
import pandas as pd
import pickle


In [8]:
file_path = '/content/test.pkl'
with open(file_path, 'rb') as file:
    test = pickle.load(file)

file_path = '/content/train.pkl'
with open(file_path, 'rb') as file:
    train = pickle.load(file)

file_path = '/content/val.pkl'
with open(file_path, 'rb') as file:
    val = pickle.load(file)

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.6 MB/s eta 0:00:00


In [7]:
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf
from tqdm import tqdm

marbert_model_path = 'UBC-NLP/MARBERT'
tokenizer = AutoTokenizer.from_pretrained(marbert_model_path, from_tf=True)
marbert_model = TFAutoModel.from_pretrained(marbert_model_path, output_hidden_states=True)

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at UBC-NLP/MARBERT.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


# per word embedding

In [10]:
remove_special_tokens=0  #change this to 0 if you want to keep the special token
stemtype = 'Light Stemming'


def bert_tokenize(text: str) -> dict:
  tokens = tokenizer(text, padding='max_length', truncation=True, max_length=50)
  if remove_special_tokens == 1:
    shape = np.array(tokens['input_ids']).shape
    modified_input_ids = np.zeros(shape).astype(np.int32)
    modified_attention_mask = np.zeros(shape).astype(np.int32)
    # Modify the input IDs and attention mask as per your requirement
    for i in range(shape[0]):
      modified_input_ids[i] = [0 if token_id == 1 else 0 if token_id == 3 else 0 if token_id == 0 else 0 if token_id == 2 else 0 if token_id == 4 else token_id for token_id in tokens['input_ids'][i]]
      modified_attention_mask[i] = [0 if token_id in [1, 3, 0, 2, 4] else 1 for token_id in tokens['input_ids'][i]]
    # Update the input IDs and attention mask in the tokens dictionary
    tokens['input_ids'] = modified_input_ids
    tokens['attention_mask'] = modified_attention_mask
  return tokens


def get_embeddings(ids, mask, type_ids):
  ids = tf.convert_to_tensor(ids)
  #print(ids.shape)
  mask = tf.convert_to_tensor(mask)
  #print(mask.shape)
  #print(mask)
  type_ids = tf.convert_to_tensor(type_ids)
  #print(type_ids.shape)
  hidden_states = marbert_model(input_ids=ids, attention_mask=mask, token_type_ids=type_ids)[0]
  averaged_embedding = tf.reduce_mean(hidden_states, axis=1)
  return hidden_states.numpy()

def embedd(text):
  text=text.values.astype(str).tolist()
  tokens = bert_tokenize(text)
  xlen = np.array(tokens['input_ids']).shape[0]
  x_emb = np.zeros((xlen,50,768))
  for i in range(0,xlen,100):
    if(i+100 < xlen):
        input_ids = tokens['input_ids'][i:i+100]
        attention_mask = tokens['attention_mask'][i:i+100]
        token_type_ids = tokens['token_type_ids'][i:i+100]
        x_emb[i:i+100] = get_embeddings(input_ids,attention_mask,token_type_ids)
    else:
        input_ids = tokens['input_ids'][i:xlen]
        attention_mask = tokens['attention_mask'][i:xlen]
        token_type_ids = tokens['token_type_ids'][i:xlen]
        x_emb[i:xlen] = get_embeddings(input_ids,attention_mask,token_type_ids)
  return x_emb

In [11]:
train_embedd=embedd(train["Light Stemming"])
val_embedd=embedd(val["Light Stemming"])
test_embedd=embedd(test["Light Stemming"])

# Avreage over axis 1 to get the embedding of the whole sentence

In [12]:
test_whole=np.mean(test_embedd, axis=1)

In [13]:
train_whole=np.mean(ls_train_embeddings_words, axis=1)

In [14]:
val_whole=np.mean(val_embedd, axis=1)

# save the embedded data in pickle files

In [15]:
with open('train_embeddings_per_word.pkl', 'wb') as f:
    pickle.dump(train_embedd, f)
with open('test_embeddings_per_word.pkl', 'wb') as f:
    pickle.dump(test_embedd, f)
with open('val_embeddings_per_word.pkl', 'wb') as f:
    pickle.dump(val_embedd, f)

with open('test_embeddings_per_sentence.pkl', 'wb') as f:
    pickle.dump(test_whole, f)
with open('train_embeddings_per_sentence.pkl', 'wb') as f:
    pickle.dump(train_whole, f)
with open('val_embeddings_per_sentence.pkl', 'wb') as f:
    pickle.dump(val_whole, f)

# move the data to the drive if you wanna save

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [16]:
import shutil

# Specify the paths for the source pickle files
source_files = [
    'train_embeddings_per_word.pkl',
    'test_embeddings_per_word.pkl',
    'val_embeddings_per_word.pkl',
    'test_embeddings_per_sentence.pkl',
    'train_embeddings_per_sentence.pkl',
    'val_embeddings_per_sentence.pkl'
]

# Destination path in your Google Drive
destination_folder = '/content/drive/MyDrive/data/emotion_data_tweets/'

# Move each file to the destination folder
for file in source_files:
    source_path = '/content/' + file
    destination_path = destination_folder + file
    shutil.move(source_path, destination_path)
